In [86]:
import pandas as pd
import matplotlib.pyplot as plt
import csv
import numpy as np
import seaborn as sns
from numpy import loadtxt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix , classification_report, precision_score, accuracy_score
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LeakyReLU,PReLU,ELU
from keras.layers import Dropout
from sklearn.model_selection import StratifiedKFold as kfold

#**Loading Data**

In [11]:
from google.colab import files
uploaded = files.upload()

Saving nn_data.txt to nn_data (1).txt


In [ ]:
data = pd.read_csv(io.BytesIO(uploaded['nn_data.txt']), header=None)
print(data.head())

In [42]:
x_unscaled = data.iloc[:,0:8]
y_unscaled = data.iloc[:,8]

# **Scaling Data**

In [ ]:
scaled_data_object = MinMaxScaler().fit(data)
scaled_data = scaled_data_object.transform(data)
scaled_data

In [44]:
x_scaled = scaled_data[:,0:8]
y_scaled = scaled_data[:,8]

# **Neural Network Model Without K-Fold Validation**

In [105]:
def ann_model_without_k_fold(input,output,epochs,batch_size):
  X_train, X_test, y_train, y_test = train_test_split(input, output, test_size=0.3)
  model = Sequential()
  model.add(Dense(8, input_shape=(8,), activation='relu'))
  model.add(Dense(6, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size)
  y_pred = (model.predict(X_test) > 0.5).astype(int)
  precisionScore = precision_score(y_test, y_pred)
  accuracyScore = accuracy_score(y_test, y_pred)
  return precisionScore,accuracyScore

# **Initializing K-Fold**

In [92]:
k = 10
folds = kfold(n_splits=k)
folds

StratifiedKFold(n_splits=10, random_state=None, shuffle=False)

In [93]:
kfold_precision_scores = []
kfold_accuracy_scores = []

# **Neural Network Model With K-Fold Validation**

In [98]:
def ann_model_with_k_fold(train_input,train_output,test_input,test_output,epochs,batch_size):
  model = Sequential()
  model.add(Dense(8, input_shape=(8,), activation='relu'))
  model.add(Dense(6, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  model.fit(train_input, train_output, epochs=epochs, batch_size=batch_size)
  y_pred = (model.predict(test_input) > 0.5).astype(int)
  kfold_precision_scores.append(precision_score(test_output, y_pred))
  kfold_accuracy_scores.append(accuracy_score(test_output, y_pred))

# **Precision & Accuracy for "Unscaled Dataset" , "Without K-Fold" Neural Network Model**

In [106]:
unscaled_precision_score,unscaled_accuracy_score = ann_model_without_k_fold(x_unscaled,y_unscaled,100,10)
print('Precision: %.2f' % (unscaled_precision_score*100))
print('Accuracy: %.2f' % (unscaled_accuracy_score*100))

Epoch 1/100
54/54 [==============================] - 1s 2ms/step - loss: 71.8190 - accuracy: 0.3594
Epoch 2/100
54/54 [==============================] - 0s 2ms/step - loss: 34.4613 - accuracy: 0.3594
Epoch 3/100
54/54 [==============================] - 0s 2ms/step - loss: 11.5836 - accuracy: 0.4507
Epoch 4/100
54/54 [==============================] - 0s 1ms/step - loss: 5.0587 - accuracy: 0.5605
Epoch 5/100
54/54 [==============================] - 0s 1ms/step - loss: 4.2988 - accuracy: 0.5754
Epoch 6/100
54/54 [==============================] - 0s 2ms/step - loss: 3.7792 - accuracy: 0.5773
Epoch 7/100
54/54 [==============================] - 0s 2ms/step - loss: 3.3517 - accuracy: 0.5922
Epoch 8/100
54/54 [==============================] - 0s 2ms/step - loss: 2.9864 - accuracy: 0.6052
Epoch 9/100
54/54 [==============================] - 0s 1ms/step - loss: 2.6221 - accuracy: 0.5866
Epoch 10/100
54/54 [==============================] - 0s 1ms/step - loss: 2.3388 - accuracy: 0.5791
Epoch 

# **Precision & Accuracy for "Scaled Dataset" , "Without K-Fold" Neural Network Model**

In [107]:
scaled_precision_score,scaled_accuracy_score = ann_model_without_k_fold(x_scaled,y_scaled,100,10)
print('Precision: %.2f' % (scaled_precision_score*100))
print('Accuracy: %.2f' % (scaled_accuracy_score*100))

Epoch 1/100
54/54 [==============================] - 1s 1ms/step - loss: 0.6902 - accuracy: 0.6387
Epoch 2/100
54/54 [==============================] - 0s 1ms/step - loss: 0.6833 - accuracy: 0.6611
Epoch 3/100
54/54 [==============================] - 0s 1ms/step - loss: 0.6727 - accuracy: 0.6518
Epoch 4/100
54/54 [==============================] - 0s 2ms/step - loss: 0.6602 - accuracy: 0.6462
Epoch 5/100
54/54 [==============================] - 0s 1ms/step - loss: 0.6457 - accuracy: 0.6499
Epoch 6/100
54/54 [==============================] - 0s 1ms/step - loss: 0.6326 - accuracy: 0.6462
Epoch 7/100
54/54 [==============================] - 0s 2ms/step - loss: 0.6230 - accuracy: 0.6499
Epoch 8/100
54/54 [==============================] - 0s 1ms/step - loss: 0.6146 - accuracy: 0.6629
Epoch 9/100
54/54 [==============================] - 0s 1ms/step - loss: 0.6069 - accuracy: 0.6704
Epoch 10/100
54/54 [==============================] - 0s 1ms/step - loss: 0.5992 - accuracy: 0.6797
Epoch 11/

# **Precision & Accuracy for "Scaled Data" , "K-Fold" Neural Network Model**

In [108]:
for i, j in folds.split(x_scaled,y_scaled):
   train_x, test_x = x_scaled[i], x_scaled[j]
   train_y, test_y = y_scaled[i], y_scaled[j]
   ann_model_with_k_fold(train_x, train_y, test_x, test_y, 100, 10)

Epoch 1/100
70/70 [==============================] - 1s 1ms/step - loss: 0.6735 - accuracy: 0.6353
Epoch 2/100
70/70 [==============================] - 0s 1ms/step - loss: 0.6595 - accuracy: 0.6512
Epoch 3/100
70/70 [==============================] - 0s 1ms/step - loss: 0.6547 - accuracy: 0.6512
Epoch 4/100
70/70 [==============================] - 0s 1ms/step - loss: 0.6503 - accuracy: 0.6512
Epoch 5/100
70/70 [==============================] - 0s 1ms/step - loss: 0.6466 - accuracy: 0.6512
Epoch 6/100
70/70 [==============================] - 0s 1ms/step - loss: 0.6420 - accuracy: 0.6512
Epoch 7/100
70/70 [==============================] - 0s 1ms/step - loss: 0.6375 - accuracy: 0.6512
Epoch 8/100
70/70 [==============================] - 0s 1ms/step - loss: 0.6322 - accuracy: 0.6512
Epoch 9/100
70/70 [==============================] - 0s 1ms/step - loss: 0.6265 - accuracy: 0.6512
Epoch 10/100
70/70 [==============================] - 0s 2ms/step - loss: 0.6214 - accuracy: 0.6512
Epoch 11/

In [109]:
print("Avg Precision :  %.2f%% " % (np.mean(kfold_precision_scores)*100))
print("Avg Accuracy : %.2f%% " % (np.mean(kfold_accuracy_scores)*100))

Avg Precision :  72.54% 
Avg Accuracy : 76.83% 


# **Results**


1. For Unscaled Dataset without K-fold Validation 
  *   Accuracy : 70.13%
  *   Precision : 54.84%


2.   For Scaled Dataset without K-fold Validation
  *   Accuracy : 77.06%
  *   Precision : 69.23%

3. For Scaled Dataset with K-fold Validation
  *   Accuracy : 76.83%
  *   Precision : 72.54%